<a href="https://colab.research.google.com/github/asokraju/Rienforcement-Learning/blob/master/David_Silver_Lecture_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 4 : Model free prediction

This Document contains the simulation of examples provided in [Lecture 4](https://youtu.be/PnHCvfgC_ZA?list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ) of Introduction to Rienforcement Learning by David Silver

Author: Krishna Chaitanya Kosaraju

email: kkrishnachaitanya89@gmail.com

# Part one : Monte-Carlo Learning

In the earlier methods, such as Value iteration and Policy iteration, we need to know the complete MDP. However, MC learning we draw samples (actually episodes) from the real system. We use these episodes to compute the returns, using which we update the estimate the Value function (for the current policy).

Note: The aim is to compute the Value function for a given policy.

$V(s) = V(s) + \dfrac{1}{N(s)} * (G_t - V(s))$

In [0]:
#!pip install gym

import numpy as np
import gym

env = gym.make('Blackjack-v0')


OpenAI Gym’s Blackjack-v0

To play Blackjack, a player obtains cards that total as close to 21 without going over. Face cards (K, Q, J) are each worth ten points. An Ace can be counted as either 1 or 11 points. If 11, it’s considered a usable ace.

The game starts with the player and dealer each receiving two cards, with one card face up. The player can choose to get more cards by asking for a HIT,  or stop getting more cards by asking to STICK, or they may BUST by going over 21. After the player has decided to STICK, the dealer will then draw cards until reaching a sum of 17 or greater.

In OpenAI’s Gym a state in Blackjack has three variables: the sum of cards in the player’s hand, the card that the dealer is showing, and whether or not the player has a usable ace. The cards are dealt from an infinite deck.

Whoever is closer to 21 when the game is over is the winner. Rewards are +1 if the player wins, 0 if there is a draw, and -1 if the player loses.

In [0]:
# env.reset() will reset the environment and give us the first state
# observation or state is player sum of first two cards, dealers score (we only know his card) and do we have an ace?
state = env.reset()

def report_score(state):
  sum_of_cards, dealer_score, usable_ace = state
  print("player score is {}, the dealer score is {}, and ACE is {}".format(sum_of_cards, dealer_score, usable_ace))

# Actions can take two values 0, and 1. Action = 0 implies STICK and Action = 1 implies HIT.
# As given in Lecture we choose an greedy policy
def greedy_policy(sum_of_cards):
  if sum_of_cards >= 20:
    return 0
  else:
    return 1

# The following function will help us compute the Returns
def Returns(rewards, gamma, States):
  """
  The following function will help us compute the Returns
  Args:
      sequence of rewards, discount factor, and sequence of states
  output:
        numpy array of returns, dictionary with state and rewards
  """
  length = rewards.shape[0]
  temp = np.full(length+1, 0.0)
  for i, x in enumerate(reversed(rewards)):
    temp[length - i-1] = x+gamma*temp[length-i]
  dict_out = {}
  for i in range(length):
    dict_out[States[i]] = temp[i]
  return temp[:length], dict_out





episodes = 1000*500
Value_function = {}
Counter = {}
for eps in range(episodes):
  Reward = np.array([]) #holds sequence of rewards
  States = [] # holds sequence of states
  Actions = [] # holds sequence of actions
  state = env.reset()
  #Value_function = 
  #Counter = 
  for i in range(100):
    #report_score(state)
    sum_of_cards, dealer_score, usable_ace = state
    States.append(state)
    Actions.append(greedy_policy(sum_of_cards))
    state, r, done, _ = env.step(Actions[i])
    Reward = np.append(Reward,r)
    if done:
      break
  # We now have the complete information of an episode (we reached the terminal state).
  # we use this to compute the Returns (G_t).
  Epi_Returns, dict_returns = Returns(Reward, gamma= 0.9, States= States)
  for state in States:
    if state in Value_function.keys():
      # if we already have a estimate for the value function at this particular state, 
      #then we update the counter which measures noumber of times it explored the state (in this episode)
      Counter[state] = Counter[state] + 1
      #Since we have the information on the complete episode, we can compute the net Returns (G_t) for each visited state in the episode.
      # Then we update the Value function at this state towards its Net return, i.e.
      # V = V + (1/N) * (G_t - V)
      Value_function[state] = Value_function[state] + (1/Counter[state])*(dict_returns[state] - Value_function[state])
    else:
      # However, if we do not have an esimate of the value funtion at this state, 
      #we can store the computed Return (G_t) as an estimate for Value function, that can be used latter.
      Value_function[state] = dict_returns[state]
      Counter[state] = 1



In [0]:
#Spliting the Value function into an useful numpy arrays (one for Ace_card =True and Ace_card = False)
V_a = []
V_na = []
Reward = np.append(Reward,r)
for state in Value_function:
  sum_of_cards, dealer_score, usable_ace = state
  if usable_ace == True:
    V_a.append([sum_of_cards, dealer_score, Value_function[state]])
  else:
    V_na.append([sum_of_cards, dealer_score, Value_function[state]])
V_a_np = np.asarray(V_a, dtype=np.float32)
V_na_np = np.asarray(V_na, dtype=np.float32)

# Part two : Temporal Difference Learning

In  MC learning, to update the Value function we need to compute the net returns of the episode, i.e., $G_t$. To compute this, we need to wait till the end of the episode. This is often not desirable.

1. Note that if V represent the true Value function  (which we do not know), then ($G_t$) reprsent its sample. 
2. How much variance does $G_t$ it have? We computed $G_t$ from various different realworld noisy samples. Hence, $G_t$ can have a very high variance but unbiased.
3. Hence we take the average over all the samples ($G_t$) which makes it a good estimate of the Value function. However, it still have a good amount of variance.


$V(s_{t}) = V(s_t) +  \alpha (\underbrace{R_{t+1} + \gamma V(s_{t+1})- V(s_t)}_{TD - error})$

TD - Targer = $R_{t+1} + \gamma V(s_{t+1})$



In [0]:
# We use the same greedy policy as provided in the above section 
# And find its corresponding value function using TD(0)

no_of_episodes = 1000*1000
alpha = .01 #learning rate, we will change it latter
counter = {}
gamma = 1 # discount factor
V = {} # placeholder for value function
for i in range(no_of_episodes):
  alpha = 1/(i+1)
  #Let us reset the environment
  state = env.reset()
  for i in range(100):
    sum_of_cards, dealer_score, usable_ace = state
    action = greedy_policy(sum_of_cards)
    next_state, reward, done, _ = env.step(action)
    if state not in V.keys():
      V[state] = 0 #setting the inital estimate of value function at the current explored state  to zero
                   # This makes the TD learning biased estimate. 
                   # however, low variance, since it 
    # we now have the seqence state, action, reward, next_state
    #if state not in counter.keys():
    #  counter[state] = 1
    #else:
    #  counter[state] = 1 + counter[state]
    if next_state not in V.keys():
      V[next_state] = 0 #setting the inital estimate of value function at the newly explored state  to zero
      V[state] = V[state] + alpha * (reward + gamma * V[next_state] - V[state])
      #V[state] = V[state] + (1/counter[state]) * (reward + gamma * V[next_state] - V[state])
    else:
      V[state] = V[state] + alpha * (reward + gamma * V[next_state] - V[state])
      #V[state] = V[state] + (1/counter[state]) * (reward + gamma * V[next_state] - V[state])
    state = next_state
    if done:
      break
    

print(V)
print(counter)

{(6, 6, False): -0.0007605008538500138, (17, 6, True): -0.002144800717241569, (17, 6, False): -0.6795454303074674, (27, 6, False): 0, (14, 7, False): -0.03459524876680953, (20, 7, False): 0.6427252836551335, (9, 6, False): -0.3224523902458162, (18, 6, False): -0.38159955142467294, (25, 6, False): 0, (15, 10, False): -0.35221800979351875, (22, 10, False): 0, (12, 3, False): -0.04252110577237401, (18, 3, False): -0.24495956420750276, (28, 3, False): 0, (16, 10, False): -0.21971532485519493, (18, 10, False): -0.4325483109209419, (24, 10, False): 0, (16, 1, False): -0.18528682209124595, (23, 1, False): 0, (12, 4, False): -0.029028721422614288, (13, 4, False): -0.1592562437782069, (23, 4, False): 0, (20, 10, True): 0.14277973389400594, (9, 2, False): -0.0023399288557163136, (19, 2, False): -0.17919926651819218, (29, 2, False): 0, (16, 5, False): -0.08037800158836882, (20, 5, False): 0.18705988277524846, (14, 2, False): -0.1557073576897336, (23, 2, False): 0, (10, 10, False): -0.009622843449

# Part three : Forward TD($\lambda$)
 Actually it should be TD$(\lambda, n)$, n is ofted ommitted!

 1. n-represents how many steps do we consider.
 2. For example n=1 represents TD algorithm. However, n= $\infty$ (or Termianal state) represents the MC algoritham.
 3. What $\lambda$ does is (geometrically) wieghted average the retures over all possible $n$.
 4. If we compute returns, we get 
  $G_t^n = R_t + \gamma R_{t+1} +\cdots + \gamma^n R_{t+n}$. 
 5.  $G_t^{\lambda} = (1-\lambda)\sum_{i=1}^{n}\lambda^{n-1}G_t^n$

Note: What $\lambda$ does is that it gives high priority to the states that nearby (1, 2, 3, ...) than the terminal state (..., n-2, n-1, n). This helps in reducing the variance cause by the noise (for using the future state) and reducing the bias in TD for not using the information of the future states.

Note: $0\leq \lambda < 1$

1. $n=T$ (Terminal state) and $\lambda =0$ implies we are using MC algorithm
2.$n=1$ and $\lambda =0$ implies TD Algorithm.

In [0]:
def Lambda_Returns(rewards, gamma, States, lam):
  """
  The following function will help us compute the lambda discounted Returns
  Args:
      sequence of rewards, discount factor, and sequence of states, lambda discount factor
  output:
        numpy array of lambda-returns, dictionary with state and lambda-rewards
  """
  G, _ = Returns(rewards, gamma, States)
  length = rewards.shape[0]
  temp = np.full(length+1, 0.0)
  for i, x in enumerate(reversed(G)):
    temp[length - i-1] = x+lam*temp[length-i]
  dict_out = {}
  for i in range(length):
    dict_out[States[i]] = (1-lam)*temp[i]
  return (1-lam)*temp[:length], dict_out



episodes = 1000*500
Value_function = {}
Counter = {}
for eps in range(episodes):
  Reward = np.array([]) #holds sequence of rewards
  States = [] # holds sequence of states
  Actions = [] # holds sequence of actions
  state = env.reset()
  #Value_function = 
  #Counter = 
  for i in range(100):
    #report_score(state)
    sum_of_cards, dealer_score, usable_ace = state
    States.append(state)
    Actions.append(greedy_policy(sum_of_cards))
    state, r, done, _ = env.step(Actions[i])
    Reward = np.append(Reward,r)
    if done:
      break
  # We now have the complete information of an episode (we reached the terminal state).
  # we use this to compute the Returns (G_t).
  Epi_Returns, dict_returns = Lambda_Returns(Reward, gamma= 0.9, States= States, lam =0.5)
  for state in States:
    if state in Value_function.keys():
      # if we already have a estimate for the value function at this particular state, 
      #then we update the counter which measures noumber of times it explored the state (in this episode)
      Counter[state] = Counter[state] + 1
      #Since we have the information on the complete episode, we can compute the net Returns (G_t) for each visited state in the episode.
      # Then we update the Value function at this state towards its Net return, i.e.
      # V = V + (1/N) * (G_t^lam - V)
      Value_function[state] = Value_function[state] + (1/Counter[state])*(dict_returns[state] - Value_function[state])
    else:
      # However, if we do not have an esimate of the value funtion at this state, 
      #we can store the computed Return (G_t) as an estimate for Value function, that can be used latter.
      Value_function[state] = dict_returns[state]
      Counter[state] = 1



#Part four : Backward TD($\lambda$) and Eligibility Traces

In [0]:
def greedy_policy(sum_of_cards):
  if sum_of_cards >= 20:
    return 0
  else:
    return 1


episodes = 1000*500
Value_function = {}
lam =0.5
alpha = 0.05
gamma = 0.9
for eps in range(episodes):
  ET = {}
  #Reward = np.array([]) #holds sequence of rewards
  States = [] # holds sequence of states
  #Actions = [] # holds sequence of actions
  state = env.reset()
  #Value_function = 
  #Counter =
  done = False 
  for i in range(100):
    #
    if done:
      break
    if state not in ET.keys():
      ET[state] = 1
    else:
      if state not in States:
        ET[state] = lam*gamma*ET[state] + 0
      else:
        ET[state] = lam*gamma*ET[state] + 1
    sum_of_cards, dealer_score, usable_ace = state
    States.append(state)
    #print("state",state)
    #print("policy",greedy_policy(sum_of_cards))
    new_state, r, done, _ = env.step(greedy_policy(sum_of_cards))
    #Reward = np.append(Reward,r)
    #print("new_state",new_state)
    #print("reward",r)
    if state not in Value_function.keys():
      Value_function[state] = 0
    if new_state not in Value_function.keys():
      Value_function[new_state] = 0
    delta = r + gamma*Value_function[new_state] - Value_function[state]
    #print("delta", delta)
    #print("reward",r)
    Value_function[state] = Value_function[state] + alpha*delta*ET[state]
    state = new_state

Value_function


{(4, 1, False): 0.2608627770309068,
 (4, 2, False): 1.2097616115943441,
 (4, 3, False): 1.373249142812497,
 (4, 4, False): 1.3118009051624495,
 (4, 5, False): 1.1451075227309846,
 (4, 6, False): 1.2311946354871461,
 (4, 7, False): 1.394866759390313,
 (4, 8, False): 1.3230475428361712,
 (4, 9, False): 1.1783879102056125,
 (4, 10, False): 0.8307176859376866,
 (5, 1, False): 0.13923643537540087,
 (5, 2, False): 0.9913091206863692,
 (5, 3, False): 1.011880841916401,
 (5, 4, False): 1.5590100739761643,
 (5, 5, False): 1.00209754971677,
 (5, 6, False): 1.0881517147829087,
 (5, 7, False): 1.3375760182141492,
 (5, 8, False): 1.2540069029068375,
 (5, 9, False): 1.3922984923429302,
 (5, 10, False): 0.5405805891593529,
 (6, 1, False): 0.17898050738212737,
 (6, 2, False): 1.1963255607232093,
 (6, 3, False): 1.2738581227918513,
 (6, 4, False): 1.1143876612598478,
 (6, 5, False): 1.1862445946341187,
 (6, 6, False): 1.1354050801532831,
 (6, 7, False): 1.4079360631669864,
 (6, 8, False): 1.05613362264